In [9]:
# import libraries
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from scipy.stats import randint
from sklearn.impute import SimpleImputer, KNNImputer
import pandas as pd
import numpy as np
import seaborn as sns
import statsmodels.api as sm
from lussi.stackoverflow import *
from lussi.ziprecruiter import *
nogit_data_dir = "622data_nogit"
raw_stack  = load_stack(data_dir = nogit_data_dir, stack_type=StackType.RAW)
wide_stack = load_stack(data_dir = nogit_data_dir, stack_type=StackType.WIDE)


,Year,OrgSize,Country,Employment,Gender,EdLevel,US_State,Age,DevType,Sexuality,...,mariadb,microsoftazure,googlecloud,ibmcloudorwatson,kubernetes,linux,windows,sexuality_grouped,ethnicity_grouped,aws
0,2017,NaN,United States,"Not employed, and not looking for work",Male,Secondary,NaN,NaN,NaN,NaN,...,no,no,no,no,no,no,no,NaN,non-minority,no
1,2017,20 to 99 employees,United Kingdom,Part-Time,Male,Bachelors,NaN,NaN,NaN,NaN,...,no,no,no,no,no,no,no,NaN,non-minority,yes
2,2017,"10,000 or more employees",United Kingdom,Full-Time,Male,Bachelors,NaN,NaN,Other,NaN,...,no,no,no,no,no,no,no,NaN,non-minority,no
3,2017,"10,000 or more employees",United States,Full-Time,Male,Doctorate,NaN,NaN,NaN,NaN,...,no,no,no,no,no,yes,yes,NaN,non-minority,yes
4,2017,10 to 19 employees,Switzerland,Full-Time,NaN,Masters,NaN,NaN,Mobile developer; Graphics programming; Deskto...,NaN,...,no,no,no,no,no,no,no,NaN,NaN,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
549477,2023,100 to 499 employees,Brazil,Full-Time,NaN,Associates,NaN,25-34 years old,"Developer, front-end",NaN,...,no,no,yes,no,no,no,no,NaN,minority,no
549478,2023,NaN,Romania,Full-Time,NaN,Bachelors,NaN,18-24 years old,"Developer, back-end",NaN,...,no,no,no,no,no,no,no,NaN,minority,no
549479,2023,NaN,Israel,I prefer not to say,NaN,Something Else,NaN,Prefer not to say,NaN,NaN,...,no,no,no,no,no,no,no,NaN,minority,yes
549480,2023,NaN,Switzerland,Part-Time,NaN,Secondary,NaN,Under 18 years old,System administrator,NaN,...,yes,yes,yes,no,no,no,no,NaN,minority,yes


QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [3]:
# Which columns are in use, and by year?
filtered_columns = wide_stack.loc[:, 'Year':'AgeAvg']
grouped_summary_filtered = filtered_columns.groupby("Year").agg(lambda x: (x.notnull().mean() * 100)).reset_index()
grouped_summary_filtered_rounded = grouped_summary_filtered.round(1)
from IPython.display import display
display(grouped_summary_filtered_rounded)

,Year,OrgSize,Country,Employment,Gender,EdLevel,US_State,Age,DevType,Sexuality,Ethnicity,DatabaseWorkedWith,LanguageWorkedWith,PlatformWorkedWith,YearsCodePro,AnnualSalary,YearsCodeProAvg,OrgSizeAvg,AgeAvg
0,2017,75.7,100.0,100.0,68.2,100.0,0.0,0.0,70.3,0.0,64.3,57.3,71.3,56.7,99.5,25.1,96.6,72.7,0.0
1,2018,72.4,99.6,96.4,65.2,95.8,0.0,65.3,93.2,60.5,58.1,67.0,79.2,66.8,94.9,48.3,94.9,72.4,65.3
2,2019,80.8,99.9,98.1,96.1,97.2,0.0,89.1,91.5,85.7,86.3,85.5,98.5,90.8,83.6,0.0,83.6,75.9,89.1
3,2020,68.8,99.4,99.1,78.4,89.1,0.0,70.5,76.6,68.2,71.3,76.8,89.0,83.5,71.9,0.0,71.9,65.4,70.5
4,2021,72.8,100.0,99.9,98.6,99.6,17.9,98.8,79.7,87.9,95.2,83.3,98.7,62.5,73.4,56.1,73.4,66.3,98.1
5,2022,69.7,98.0,97.9,96.7,97.7,0.0,96.8,83.7,90.9,94.8,82.1,96.9,68.1,70.7,52.0,70.7,64.5,96.1
6,2023,72.9,98.6,98.6,0.0,98.6,0.0,100.0,86.2,0.0,0.0,82.3,97.7,71.3,74.2,53.8,74.2,66.8,99.5


In [2]:
## which features are important, assuming that we are trying to predict salary.

from statsmodels.stats.outliers_influence import variance_inflation_factor

# Calculate VIF for each feature
vif = pd.DataFrame()
vif["features"] = X_train.columns
vif["VIF"] = [variance_inflation_factor(X_train.values, i) for i in range(X_train.shape[1])]
print(vif)

# none with VIF above 10, so think this is OK. 

# model without randomizedsearchcv

import xgboost as xgb
from sklearn.metrics import mean_squared_error as MSE
# Instantiation 
xgb_r = xgb.XGBRegressor(objective='reg:squarederror', 
                         n_estimators=100, 
                         learning_rate=0.1, 
                         max_depth=6, 
                         enable_categorical=True,  # Enable categorical feature handling
                         seed=123) 
  
# Fitting the model 
xgb_r.fit(X_train, Y_train_imputed) 
  
# Predict the model 
pred = xgb_r.predict(X_test) 

# Assuming Y_test is a pandas Series - we're only imputing the test to compute RMSE
Y_test_imputed = Y_test.copy()  # Create a copy to avoid modifying the original
Y_test_imputed.fillna(Y_test.mean(), inplace=True)  # Mean imputation

# compute RMSE
rmse = np.sqrt(mean_squared_error(Y_test_imputed, pred)) 
print("RMSE : % f" % (rmse))


NameError: name 'X_train' is not defined